# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of native and semantic functions that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

From our own testing, planner works best with more powerful models like `gpt4` but sometimes you might get working plans with cheaper models like `gpt-35-turbo`. We encourage you to implement your own versions of the planner and use different models that fit your user needs.  

Read more about planner [here](https://aka.ms/sk/concepts/planner)

In [1]:
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv()

# Get the value of AZURE_OPENAI_DEPLOYMENT_GPT4
azure_openai_deployment_gpt4 = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT4")


In [2]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, AzureChatCompletion

kernel = sk.Kernel()

deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment_name=azure_openai_deployment_gpt4, endpoint=endpoint, api_key=api_key))


# Basic Planner

# The Plan Object Model

# Sequential Planner

# Stepwise Planner

Stepwise Planner is based off the paper from MRKL (Modular Reasoning, Knowledge and Language) and is similar to other papers like ReACT (Reasoning and Acting in Language Models). At the core, the stepwise planner allows for the AI to form "thoughts" and "observations" and execute actions based off those to achieve a user's goal. This continues until all required functions are complete and a final output is generated.

See a video walkthrough of Stepwise Planner [here.](https://youtu.be/DG_Ge1v0c4Q?si=T1CHaAm1vV0mWRHu)

In [3]:
from semantic_kernel.planning import StepwisePlanner
from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import (
    StepwisePlannerConfig,
)

Let's create a Bing Search native skill that we can pass in to the Kernel.

Make sure you have a Bing Search API key in your `.env` file

(https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)

In [4]:
from semantic_kernel.connectors.search_engine import BingConnector
from semantic_kernel.core_skills import WebSearchEngineSkill

BING_API_KEY = sk.bing_search_settings_from_dot_env()
connector = BingConnector(BING_API_KEY)
kernel.import_skill(WebSearchEngineSkill(connector), skill_name="WebSearch")

{'searchAsync': SKFunction()}

In [5]:
result = await connector.search_async("How to make a cake", 2,0)

for i, r in enumerate(result):
    print(i, r)

0 Learn how to make a basic vanilla cake with plain flour, baking powder, sugar, butter, eggs, milk and vanilla extract. Find the ingredients list, method, variations and reader ratings for this easy and moist dessert.
1 Learn how to mix, prep and bake a cake with this easy guide from Food Network. Follow the tips and watch the video for success every time you bake. Find out how to measure ingredients, preheat the oven, use parchment paper, line the pan, rotate the pan and more.


In [6]:
search_func = kernel.skills.get_function("WebSearch", "searchAsync")

search_context = sk.ContextVariables(
    # content="semantic kernel",
    # content = "length of the Burj Khalifa",
    content = "how to make a cake?",
    variables={
        "num_results": "10",
    }
)
result = await search_func.invoke_async(
    variables=  search_context
    )


for i, r in enumerate(eval(result.result)):
    print(i, r)

0 Preheat oven to 160°C (325°F). Place the flour, baking powder, sugar, butter, eggs, milk and vanilla in a large bowl and mix until well combined. Pour into a lightly greased tin of preferred size lined with non-stick baking paper and bake for specified time (see tin variations below) or until cooked when tested with a skewer.
1 Only fill the cake pan to 2/3 high. Use the center rack of the oven for even cooking. Rotate the Pan. Halfway through the cooking time, rotate the pan 180 degrees. Done. The cake is done when it's ...
2 2. Preheat the oven to 340 °F (171 °C) and grease your cake pan. Use a round, square, loaf, or bundt cake pan for this recipe—the choice is up to you! Rub the inside of the pan with butter, and then sprinkle a layer of flour on top. Tap the excess flour into the trash, and turn on the oven.
3 1. Preheat oven to 180°C/350°F. Grease a deep 20cm (8-inch) square cake pan; line base and sides with baking paper. 2. Beat butter, extract and sugar in a medium bowl with

Let's also add a couple more skills

In [7]:
from semantic_kernel.core_skills.math_skill import MathSkill
from semantic_kernel.core_skills.time_skill import TimeSkill

# kernel.import_skill(TimeSkill(), "time")
# kernel.import_skill(MathSkill(), "math")

In [8]:
planner = StepwisePlanner(
    kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000)
)


Now let's do a more complicated ask that will require planner to make a call to Bing to get the latest information.

In [9]:
ask = """How many total championships combined do the top 5 teams in the NBA have? if doing a web search, get top 10 results."""


plan = planner.create_plan(goal=ask)


In [10]:
result = await plan.invoke_async()

Variable `$agent_scratch_pad` not found


In [11]:
print(result)

The combined total championships of the Golden State Warriors (6), Los Angeles Lakers (17), Boston Celtics (17), Chicago Bulls (6), and San Antonio Spurs (5) is 51.


Let's see the steps that the AI took to get to the answer.

In [12]:
for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:",step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print( "  Output:\n", str.replace(result[step._outputs[0]],"\n", "\n  "))

Step: 0
Description: Execute a plan
Function: StepwisePlanner.ExecutePlan
  Output:
 This was my previous work (but they haven't seen any of it! They only see what I return as final answer):
  [THOUGHT]
  To answer this question, I need to first identify the top 5 teams in the NBA. Then, I need to find out how many championships each of these teams has won. I can use the WebSearch.searchAsync function to perform a web search for this information. Let's start by identifying the top 5 teams in the NBA.
  [ACTION]
  {"action": "WebSearch.searchAsync", "action_variables": {"input": "top 5 teams in the NBA", "num_results": "10"}}
  [OBSERVATION]
  ['(1) Head-to-head won-lost percentage (2) Division leader wins tie from team not leading a division (3) Division won-lost percentage for teams in the same division (4) Conference won-lost...', 'NBA Standings - 2023-24 season - ESPN Full Scoreboard » ESPN Visit ESPN for the complete 2023-24 NBA season standings. Includes league, conference and div

In [13]:
search_func = kernel.skills.get_function("WebSearch", "searchAsync")

In [14]:
result = await search_func.invoke_async("top 5 teams in the NBA, get top 5 web search results")
print(result)

['Visit ESPN for the complete 2023-24 NBA season standings. Includes league, conference and division standings for regular season and playoffs.']
